In [ ]:
# import pandas as pd
# from tqdm import tqdm

# df_polarizability = pd.read_csv('../polygraphpy/data/polarizability_data.csv')

# for _, row in tqdm(df_polarizability.iterrows()):
#     id_val = row['id']
#     chain_size = row['chain_size']
#     value = row['static_polarizability']
    
#     if chain_size == 0:
#         filename = f"xyz_files/monomer_{id_val}.pdb"
#     else:
#         filename = f"xyz_files/homopoly_{id_val}_chain_{chain_size}.pdb"
    
#     try:
#         with open(filename, 'r') as f:
#             content = f.read()
        
#         remark = f"REMARK static_polarizability {value}\n"
#         new_content = remark + content
        
#         with open(filename, 'w') as f:
#             f.write(new_content)
#     except:
#         pass

26582it [00:10, 2598.49it/s]


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import os

from huggingface_hub import login; 

# Dataset
class PDBDataset(Dataset):
    def __init__(self, pdb_dir, tokenizer):
        self.tokenizer = tokenizer
        self.data = []
        pdb_files = [f for f in os.listdir(pdb_dir) if f.endswith('.pdb')]
        print(f'Dataset len: {len(pdb_files)}')
        for file in pdb_files:
            full_path = os.path.join(pdb_dir, file)
            with open(full_path, 'r') as f:
                lines = f.readlines()
                prop_line = next((line for line in lines if line.startswith('REMARK')), None)
                if prop_line:
                    parts = prop_line.split()
                    prop_name = parts[1]
                    prop_value = parts[2]
                    prop = f"{prop_name}: {prop_value}"
                else:
                    prop = "Unknown: 0.0"  # Fallback
                pdb_text = ''.join(lines)
            text = f"Property: {prop}\n{pdb_text}"
            self.data.append(tokenizer(text, return_tensors='pt')['input_ids'])

    def __len__(self): return len(self.data)
    def __getitem__(self, idx): return self.data[idx]

/Users/gabriel/Documents/Apps/PolyGraphPy/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
gpt = 'deepseek-ai/DeepSeek-V3'

tokenizer = AutoTokenizer.from_pretrained(gpt)
model = AutoModelForCausalLM.from_pretrained(gpt)

dataset = PDBDataset('xyz_files', tokenizer)
dataloader = DataLoader(dataset, batch_size=16)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.1-8B.
403 Client Error. (Request ID: Root=1-6879d7f0-279d4a963e83397b269f05d7;7c3fcf39-9e5e-4274-a0f5-5a6dfe9d0b0b)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.1-8B/resolve/main/config.json.
Your request to access model meta-llama/Llama-3.1-8B is awaiting a review from the repo authors.